# rstoolbox & SilentFiles

@Author: 吴炜坤

@email：weikun.wu@xtalpi.com/weikunwu@163.com

rstoolbox是由RosettaCommon社区开发者jaumebonet开发的一款专门用于分析Rosett Silent文件的工具包。他可以以pandas的DataFrame的形式对Silent文件中的数据进行提取，使用其中的一些API可以非常方便地进行分析和作图。

> github: https://github.com/jaumebonet/RosettaSilentToolbox

> doc: http://jaumebonet.cat/RosettaSilentToolbox

### 一、安装

命令解决:
```
pip install rstoolbox
```

配置环境: 在\$HOME 目录下创建.rstoolbox.cfg文件。
```
cd ~
vi ~/.rstoolbox.cfg
```

.rstoolbox.cfg文件中的内容:
```
rosetta:
  compilation: macosclangrelease
  path: /Volumes/MiniTwo/bin/Rosetta/main/source/bin/
system:
  output: ./
  overwrite: false
```

关键设置:
- compilation: rosetta app的后缀名
- path: rosetta app所处的bin文件夹

In [1]:
# 初始化pyrosetta
from pyrosetta import *
init()

PyRosetta-4 2020 [Rosetta PyRosetta4.conda.mac.cxx11thread.serialization.python36.Release 2020.50+release.1295438cd4bd2be39c9dbbfab8db669ab62415ab 2020-12-12T00:30:01] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.cxx11thread.serialization.python36.Release r274 2020.50+release.1295438cd4b 1295438cd4bd2be39c9dbbfab8db669ab62415ab http://www.pyrosetta.org 2020-12-12T00:30:01
core.init: {0} command: PyRosetta -ex1 -ex2aro -database /opt/miniconda3/lib/python3.6/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=1375129666 seed_offset=0 real_seed=1375129666 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=1375129666 RG_type=mt19937


### 二、SilentFiles文件的获取

在第一章节，我们曾经提及过如何从Pose中输出SilentFiles以及如何在Pose中添加额外的信息，同时SimpleMetrics中的信息也会被自动储存到SilentFiles文件中。所以一个标准的流程跑下来SilentFiles应该有着所有我们需要分析的数据。

复习一下输出Silent文件以及在Pose中添加额外的信息的方法:

**以下的设计可能会耗时比较久，可以选择性跳过，结果文件已经储存在data文件夹下！**

In [1]:
# 举例使用FastDesign快速设计一些序列和结构:
from pyrosetta import pose_from_pdb, init, create_score_function
from pyrosetta.rosetta.protocols.denovo_design.movers import FastDesign
from pyrosetta.rosetta.core.pack.task import TaskFactory
from pyrosetta.rosetta.core.pose import Pose
from pyrosetta.io import poses_to_silent

# init
init('')

# load pose
starting_pose = pose_from_pdb('./data/EHEE_rd4_0976.pdb')
ref2015 = create_score_function('ref2015')
design_tf = TaskFactory()

# setup FastDesign
fastdesign = FastDesign(ref2015, 1)
fastdesign.set_default_movemap() #使用默认的Movemap()
fastdesign.set_task_factory(design_tf)

# design for 10 times: code for design pose.
# for i in range(10):
#     design_pose = Pose()
#     design_pose.assign(starting_pose)  # assign pose
#     fastdesign.apply(design_pose)  ## apply design
#     # output to silent file;
#     poses_to_silent(fastdesign, './data/design_result.silent')

PyRosetta-4 2020 [Rosetta PyRosetta4.conda.mac.cxx11thread.serialization.python36.Release 2020.50+release.1295438cd4bd2be39c9dbbfab8db669ab62415ab 2020-12-12T00:30:01] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.cxx11thread.serialization.python36.Release r274 2020.50+release.1295438cd4b 1295438cd4bd2be39c9dbbfab8db669ab62415ab http://www.pyrosetta.org 2020-12-12T00:30:01
core.init: {0} command: PyRosetta -database /opt/miniconda3/lib/python3.6/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=-1913828734 seed_offset=0 real_seed=-1913828734 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=-1913828734 RG_type=mt19937
core.chemical.GlobalResidueTypeSet:

### 三、使用rstoolbox做数据分析

上述我们已经通过分析计算了10条蛋白质的序列和结构，目前我们需要对输出的Silent文件进行分析，从中提取需要的数据并作图展示。
rstoolbox主要可用于:
- 分析序列profile
- 分析储存在silent文件中的score/其他数据，作图展示
- 比较不同参数设计的结果性质分布
- 分析、筛选以及导出满足需求的序列和结构